## Sources of redundancy
1. **Spatial redundancy**: Pixels are very similar in its neighborhood or tends to repeat textures.
2. **Temporal redundancy**: Temporally adjacent images are typically very alike.
3. **Visual redundancy**: Humans hardly perceive high spatial and temporal frequencies (we like more low frequencies).